# 여러 문서에서 찾아서 답변하는 챗봇 만들기

In [1]:
!pip install -q langchain openai tiktoken chromadb langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 692.1 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 450.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.3 MB/s eta 0:00:

In [6]:
# 크롤링한 정보를 txt로 저장한 것
!wget https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip

--2024-08-05 05:03:13--  https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chatgpt-kr/chatgpt-api-tutorial/main/ch05/data.zip [following]
--2024-08-05 05:03:14--  https://raw.githubusercontent.com/chatgpt-kr/chatgpt-api-tutorial/main/ch05/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 835816 (816K) [application/zip]
Saving to: ‘data.zip.1’

data.zip.1          100%[===================>] 816.23K  --.-KB/s    in 0.05s   

2024-08-05 05:03:14 (14.9 MB/s) - ‘data.zip.1’ saved [835816/835816]



In [7]:
!unzip data

Archive:  data.zip
replace 1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [8]:
!ls

10.txt	16.txt	21.txt	27.txt	32.txt	38.txt	43.txt	49.txt	54.txt	7.txt	       sample_data
11.txt	17.txt	22.txt	28.txt	33.txt	39.txt	44.txt	4.txt	55.txt	8.txt
12.txt	18.txt	23.txt	29.txt	34.txt	3.txt	45.txt	50.txt	56.txt	9.txt
13.txt	19.txt	24.txt	2.txt	35.txt	40.txt	46.txt	51.txt	57.txt	data.zip
14.txt	1.txt	25.txt	30.txt	36.txt	41.txt	47.txt	52.txt	5.txt	data.zip.1
15.txt	20.txt	26.txt	31.txt	37.txt	42.txt	48.txt	53.txt	6.txt	embedding.csv


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setting up LangChain

OpenAI API Key

https://platform.openai.com/account/api-keys

In [10]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from collections import Counter

## Load multiple and process documents

DirectoryLoader()는 디렉토리 내의 모든 문서들을 로드하는 역할을 합니다. 이때 glob는 해당 디렉토리 내의 어떤 확장자를 가진 파일들을 로드할 것인지를 결정합니다. 예를 들어 *.txt를 매개변수로 사용하면, 해당 디렉토리 내의 모든 txt 파일을 로드합니다. 현재 경로에 있는 모든 txt 파일을 로드하고 txt 파일의 본문은 documents라는 파이썬 리스트의 각 원소로 할당하겠습니다.

In [12]:
# loader 객체 생성
loader = DirectoryLoader('.', glob="*.txt", loader_cls=TextLoader)

documents = loader.load()

print('문서의 개수 :', len(documents))

문서의 개수 : 57


파이썬 리스트에 해당하는 documents에서 임의로 1번 문서와 21번 문서를 출력해보겠습니다.

In [13]:
print('1번 문서 :', documents[1])
print('-' * 20)
print('21번 문서 :', documents[21])
print('-' * 20)

1번 문서 : page_content='정책제목: 서울시 청년수당
정책 유형은 "진로"이며, 해당 정책은 서울시청 미래청년기획단에서 주관하고 있습니다. 이 정책은 인생전환기에 있는 청년들에게 주체적인 경로 설정 기회를 제공하여 자존감 향상과 인적 역량 강화를 목표로 합니다. 지원 내용으로는 청년의 진로 발달 단계에 따라 맞춤형 자기이해 및 진로설계 프로그램을 제공합니다. 사업은 2023년 5월 1일부터 2023년 11월 30일까지 운영되며, 신청은 2023년 4월 6일부터 2023년 4월 18일까지 가능합니다. 총 1,060명을 지원할 예정이며, 자세한 내용은 "https://youth.seoul.go.kr/site/main/content/life_desi_sch_intro"에서 확인하실 수 있습니다. 신청 자격은 19세부터 39세까지 서울에 거주하는 청년이면 누구나 신청할 수 있습니다. 학력, 전공, 취업 상태, 특화 분야에는 제한이 없으며, 추가적인 단서나 제한 대상은 없습니다. 신청 방법은 청년몽땅정보통 온라인 신청을 통해 이루어지며, 심사 및 발표는 모집 시기마다 우선 및 일반 선정 기준에 따라 진행됩니다. 제출해야 하는 서류는 일반 선정에는 없고, 우선 선정에는 13가지 요건 중 1개 이상 해당되는 자격이 필요합니다. 운영 기관은 ㈜오픈놀이며, 참고 사이트로는 "https://www.instagram.com/youthschoolseoul"과 "https://www.facebook.com/seoulyouthschool"을 참조하실 수 있습니다.' metadata={'source': '56.txt'}
--------------------
21번 문서 : page_content='정책이름: 서울희망대학장학금
서울시 교육정책과에서 주관하는 교육 정책으로, 직업전문학교에서 경제적인 걱정 없이 학업을 수행할 수 있도록 등록금을 지원하는 장학금이 있습니다. 이 정책은 2023년 3월 1일부터 2023년 12월 31일까지 운영됩니다. 학기별로 100만원이 지원되며,

documents 리스트의 각 원소의 구조를 살펴보면 page_content의 값으로 본문이 들어가고, metadata 안의 source의 값으로 해당 문서의 원본 파일의 이름이 기재되어져 있습니다.

## Split texts

ChatGPT API(GPT-3.5-turbo)는 입력으로 사용하는 텍스트와 ChatGPT API가 반환하는 답변의 길이를 모두 포함하여 최대 4,096 토큰을 처리할 수 있습니다. 예를 들어 ChatGPT API 입력으로 약 2,000 토큰 정도의 길이를 가진 입력을 사용한다면, ChatGPT는 약 2,000 토큰 내외의 답변을 할 수 있는 구조입니다. 입력의 길이가 지나치게 길 경우 ChatGPT API가 답변을 하던 도중 4,096개의 토큰을 모두 소진하면 더 이상 답변을 생성할 수 없어 텍스트가 중간에 끊긴 답변을 얻게 됩니다.  

이런 현상을 피하기 위해 너무 긴 문서들은 한 번에 입력으로 사용하는 것이 아니라 여러 개의 뭉치(Chunk)로 잘라서 ChatGPT API를 여러 번 호출하는 방식으로 해결하기도 합니다. Langchain의 RecursiveCharacterTextSplitter()는 이를 위해 사용하는 도구로 주어진 문서들을 더 작은 단위의 텍스트 뭉치(chunk)로 잘라주는 역할을 합니다.

chunk_size는 텍스트의 분할 기준이 되는 길이를 설정하고, chunk_overlap은 텍스트를 분할할 경우 텍스트의 내용을 얼만큼 겹치게 할 것인지를 의미합니다. 위에서 총 57개의 문서 파일을 로드한 documents를 입력으로 사용하여 분할된 텍스트 리스트를 texts에 저장합니다.

In [29]:
# chunk_size: 텍스트의 분할 기준이 되는 길이 설정
# chunk_overlap: 텍스트를 분할할 경우 텍스트의 내용을 얼만큼 겹치게 할 것인지 설정 (기존 텍스트에 들어간 내용이 다음 텍스트에 포함됨)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

print('분할된 텍스트의 개수 :', len(texts))

분할된 텍스트의 개수 : 64


57개의 문서가 총 64개의 텍스트로 분할되었습니다. 그 중 임의로 첫번째 문서를 출력해보겠습니다.

In [32]:
texts[0]

Document(metadata={'source': '6.txt'}, page_content='정책이름: 서울 청년포털(청년 몽땅 정보통) 운영\n서울시청 미래청년기획단에서는 사회참여를 촉진하기 위해 수요자에게 맞춤형 정책을 제공하고 편리한 청년지원사업 온라인 신청접수 시스템을 운영하여 청년의 참여를 확대하는 정책을 추진합니다. 이 프로그램은 중앙정부와 지자체별로 칸막이로 관리되는 청년정책 정보를 통합하여 연계하고, 중앙정부와 서울시 등 모든 청년정책을 통합검색 기반으로 구축합니다. 또한, 개인별 맞춤정보를 검색하고 적기에 선제적으로 제공하여 청년의 참여를 확대합니다. 신청자격은 연령, 학력, 전공, 취업 상태, 특화 분야에 대한 제한이 없습니다. 사업 운영 기간과 신청 기간은 2023년 1월 1일부터 2023년 12월 31일까지이며, 프로그램에 대한 자세한 내용은 https://youth.seoul.go.kr에서 확인하실 수 있습니다. 이 프로그램은 서울시청 미래청년기획단에서 운영됩니다. 추가로 참고할 사이트로는 http://youth.seoul.go.kr을 참고하실 수 있습니다.')

In [35]:
print('metadata:', texts[0].metadata)

metadata: {'source': '6.txt'}


In [36]:
print('page_content:', texts[0].page_content)

page_content: 정책이름: 서울 청년포털(청년 몽땅 정보통) 운영
서울시청 미래청년기획단에서는 사회참여를 촉진하기 위해 수요자에게 맞춤형 정책을 제공하고 편리한 청년지원사업 온라인 신청접수 시스템을 운영하여 청년의 참여를 확대하는 정책을 추진합니다. 이 프로그램은 중앙정부와 지자체별로 칸막이로 관리되는 청년정책 정보를 통합하여 연계하고, 중앙정부와 서울시 등 모든 청년정책을 통합검색 기반으로 구축합니다. 또한, 개인별 맞춤정보를 검색하고 적기에 선제적으로 제공하여 청년의 참여를 확대합니다. 신청자격은 연령, 학력, 전공, 취업 상태, 특화 분야에 대한 제한이 없습니다. 사업 운영 기간과 신청 기간은 2023년 1월 1일부터 2023년 12월 31일까지이며, 프로그램에 대한 자세한 내용은 https://youth.seoul.go.kr에서 확인하실 수 있습니다. 이 프로그램은 서울시청 미래청년기획단에서 운영됩니다. 추가로 참고할 사이트로는 http://youth.seoul.go.kr을 참고하실 수 있습니다.


형식은 documents 원소에서 확인하였을 때와 동일하며 page_content에는 분할된 텍스트이 본문이 저장되어져 있고, source에는 해당 본문의 원본 파일의 이름이 저장되어져 있습니다. 57개의 문서가 64개의 텍스트로 분할되었다는 것은 일부 문서는 추가적으로 분할되었음을 의미합니다.  

metadata 안의 source의 값을 이용하여 어떤 문서들을 추가적으로 분할되었는지 확인해보겠습니다. texts의 각 원소들의 source 값을 모두 체크하였을 때 source 값이 중복되어 등장한다면 해당 문서는 추가적으로 분할되었음을 의미합니다.

In [37]:
source_lst = []
for i in range(0, len(texts)):
  source_lst.append(texts[i].metadata['source'])

element_counts = Counter(source_lst)
filtered_counts = {key: value for key, value in element_counts.items() if value >= 2}
print('2개 이상으로 분할된 문서 :', filtered_counts)
print('분할된 텍스트의 개수 :', len(documents) + len(filtered_counts))

2개 이상으로 분할된 문서 : {'23.txt': 2, '22.txt': 2, '31.txt': 2, '36.txt': 2, '40.txt': 2, '49.txt': 2, '48.txt': 2}
분할된 텍스트의 개수 : 64


확인 결과 48번 문서, 23번 문서, 31번 문서, 49번 문서, 36번 문서, 22번 문서, 40번 문서가 2개의 텍스트로 추가적으로 분할되었습니다. 총 57개의 문서 중 7개의 문서가 2개의 텍스트로 분할되었음으로 총 합 64개의 텍스트가 생기게 됩니다.

## Create Chroma DB

앞서 6장에서 OpenAI Embedding API를 이용하여 텍스트를 임베딩하고, 코사인 유사도를 통해 유사한 텍스트를 가져오는 실습을 진행한 바 있습니다. Chroma DB는 이 과정들을 기능 별로 이미 구현하여 사용자가 벡터를 좀 더 쉽게 다룰 수 있도록 도와주는 편리한 벡터 응용 도구입니다. Chroma.from_documents()를 통해 벡터 도구 객체를 선언합니다. 이때 documents에는 벡터화의 단위가 될 텍스트 리스트를 매개변수로 사용하고, embedding에는 어떤 종류의 임베딩을 사용할 것인지를 기재해줍니다. 여기서는 6장에서와 같이 OpenAI의 Embedding API를 사용합니다.

In [38]:
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## Make a retriever

벡터 도구 객체를 선언하고 나면 as_retriever()를 통해서 입력된 텍스트로부터 유사한 텍스트를 찾아주는 retriever를 선언할 수 있습니다. retriever를 선언 후 get_relevant_documents()를 통해 입력된 텍스트와 유사한 문서들을 찾아서 반환합니다. 6장에서 실습했던 벡터의 유사도를 구하는 과정을 별도의 추가 구현없이 손쉽게 사용할 수 있습니다.

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("신혼 부부를 위한 정책이 있어?")
print('유사 문서 개수 :', len(docs))
print('--' * 20)
print('첫번째 유사 문서 :', docs[0])
print('--' * 20)
print('각 유사 문서의 문서 출처 :')
for doc in docs:
    print(doc.metadata["source"])

유사 문서 개수 : 4
----------------------------------------
첫번째 유사 문서 : page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.' metadata={'source': '23.txt'}
----------------------------------------
각 유사 문서의 문서 출처 :
23.txt
23.txt
40.txt
39.txt


### 결과를 k개 반환

만약 유사한 문서의 개수를 임의로 지정하고 싶다면 retriever 선언 시에 search_kwargs={"k": <문서의 개수>}를 추가적으로 입력으로 넣어주면 됩니다. 다음의 코드는 유사한 텍스트가 2개만 출력되는 모습을 보여줍니다.

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
docs = retriever.get_relevant_documents("신혼 부부를 위한 정책이 있어?")

for doc in docs:
    print(doc.metadata["source"])

23.txt
23.txt


## Make a chain

이제 ChatGPT API와 이미 만들어진 Prompt를 통해서 간단히 챗봇을 구현해봅시다. RetrievalQA.from_chain_type()의 llm 매개변수의 값으로 초기에 임포트한 OpenAI()를 사용할 경우, 기본값으로 ChatGPT API를 사용합니다.chain_type의 매개변수의 값으로 "stuff"를 사용할 경우, 사용자의 눈에는 보이지 않지만 자동으로 아래의 프롬프트를 사용하여 챗봇을 구현합니다.  

6장에서 사용자가 직접 챗봇을 구현할 때 프롬프트를 직접 작성해준 것과는 달리 여기서는 자동으로 완성되어져 있는 프롬프트를 사용합니다.
```
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{텍스트}

{질문}
```

{텍스트}에는 사용자의 질문으로부터 높은 유사도를 가진 텍스트가 들어가게 되고, {질문} 부분은 사용자의 질문이 들어가게 됩니다. retriever는 입력된 사용자의 질문으로부터 유사도를 계산하는 도구를 구현하여 매개변수의 값으로 넘겨주면 됩니다. 앞서 이미 Chroma를 통해 벡터 도구 객체로부터 구현한 retreiver를 사용합니다.  

return_source_documents는 챗봇의 답변에 사용된 텍스트들의 출처를 표시할 것인지를 의미합니다. return_source_documents의 값을 False로 할 경우, 챗봇의 답변이 어떤 텍스트에 근거하였는지 알 수 없습니다. 여기서는 뒤에서 근거가 되는 텍스트의 출처를 확인할 것이므로 True를 사용합니다.

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

## Query

이제 qa_chain을 통해 사용자의 입력으로부터 서울 청년 정책과 관련된 챗봇의 답변을 얻을 수 있습니다. 임의의 "대출과 관련된 정책이 궁금합니다"라는 텍스트를 입력하여 qa_chain의 반환 결과를 확인해봅시다.

In [ ]:
input_text = "대출과 관련된 정책이 궁금합니다"
chatbot_response = qa_chain(input_text)
print(chatbot_response)

{'query': '대출과 관련된 정책이 궁금합니다', 'result': '어떤 종류의 대출 정책에 대해 궁금하신가요? 서울시 학자금대출 신용회복 지원사업이나 역세권청년주택 주거비지원 정책에 대해 알려드릴 수 있습니다.', 'source_documents': [Document(page_content='정책내용: 서울시 학자금대출 신용회복 지원사업\n서울특별시 미래청년기획단이 주최하는 금융 정책으로, 학자금 대출로 인해 신용이 떨어져 어려움을 겪고 있는 청년층을 위한 신용회복 지원입니다. 이 정책은 분할상환약정 체결을 지원하고 초입금을 제공함으로써 신용유의자 등록을 해제하는 내용을 포함하고 있습니다. 추가적인 자부담 없이 약정을 체결할 수 있도록 합니다.\n\n지원 대상은 서울에 거주하며 학자금 대출로 인한 신용유의자인 19세부터 39세까지의 청년들이며, 약 200여명을 지원합니다. 2018년부터 2022년에 지원을 받은 사람은 2023년 지원 대상에서 제외됩니다.\n\n신청은 서울청년포털(youth.seoul.go.kr)을 통해 신청할 수 있으며, 심사 및 발표는 매월 1~2회 선정되며, 신청인원에 따라 주기가 변동할 수 있습니다.\n\n이 정책은 서울시 미래청년기획단이 운영하며, 자세한 내용은 관련 사이트를 참고하시기 바랍니다.\n\n[참고사이트]\n신용회복 신청 안내: https://youth.seoul.go.kr/site/main/board/notice/27789?baCategory1=basic&baCommSelec=true\n신청사이트: https://youth.seoul.go.kr/site/main/applyLoanCredit/step1', metadata={'source': '35.txt'}), Document(page_content='정책제목: 역세권청년주택 주거비지원\n서울시 전략주택공급과에서는 청년과 신혼부부의 주거비 부담을 경감하기 위해 임차보증금 대출을 지원하는 정책을 운영하고 있습니다. 이 정책은 혼인 감소 및 출산율 감소 문제를 해결하기

qa_chain의 결과를 chatbot_response에 저장하고 출력하면 'query', 'result', source_documents 세 개의 key값을 갖고 있습니다. 이 중 source_documents의 경우 내부적으로 앞서 확인한 형식인 page_content와 meta_data의 값을 가집니다.

'query'의 경우 현재 입력된 사용자의 질문을 의미합니다. 이때 qa_chain은 앞서 구현한 retriever을 통해서  내부적으로 '대출과 관련된 정책이 궁금합니다'와 유사도가 높은 텍스트 2개를 찾아냅니다. metadata 내의 source의 값에 따르면 해당 텍스트는 35.txt 파일과 39.txt 파일의 본문에 해당되며 두  본문(여기서는 page_content의 값)을 통해서 앞서 위에서 확인한 stuff 프롬프트를 통하여 챗봇은 답변을 작성합니다. 해당 챗봇의 답변은 결과적으로 'result'의 값에 해당됩니다.

현재 chatbot_response의 값은 너무 복잡합니다. 실제 사용자가 사용하는 챗봇을 가정하기 위해 chatbot_response로부터 실제 챗봇 답변에 해당하는 'result' 부분만 꺼내오고, 또한 답변에 사용된 출처를 확인하기 위해 metadata 내의 source의 값을 추가적으로 출력하는 get_chatbot_response() 함수를 구현합니다.

In [ ]:
def get_chatbot_response(chatbot_response):
    print(chatbot_response['result'].strip())
    print('\n문서 출처:')
    for source in chatbot_response["source_documents"]:
        print(source.metadata['source'])

"신혼 부부의 신혼집 마련을 위한 정책이 있을까?"라는 임의의 입력에 대해서 챗봇의 답변을 확인해보겠습니다.

In [ ]:
input_text = "신혼 부부의 신혼집 마련을 위한 정책이 있을까?"
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

네, 신혼 부부의 신혼집 마련을 위한 정책이 있습니다. 서울시에서는 "신혼부부 임차보증금 지원" 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 내용은 서울주거포털(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.

문서 출처:
23.txt
23.txt


신혼부부 임차 보증금 지원 정책을 소개합니다. 텍스트의 출처가 23.txt가 두 번 출력됩니다. 이는 앞서 text_splitter.split_documents()가 57개의 문서를 64개의 텍스트로 분할하는 가정에서 23.txt가 두 개의 텍스트로 분할되었기 때문입니다. chatbot_response의 값을 출력해보겠습니다.

In [ ]:
# print(chatbot_response)
chatbot_response

{'query': '신혼 부부의 신혼집 마련을 위한 정책이 있을까?',
 'result': '네, 신혼 부부의 신혼집 마련을 위한 정책이 있습니다. 서울시에서는 "신혼부부 임차보증금 지원" 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 내용은 서울주거포털(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.',
 'source_documents': [Document(page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.', metadata

출처가 23.txt로 동일하지만 두 개의 텍스트는 서로 다릅니다. 원본 파일 23.txt가 두 개의 텍스트로 분할되었기 때문입니다. 또한 두 개의 텍스트에서 **'이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.'** 라는 텍스트가 공통적으로 들어가있음을 확인할 수 있습니다. 이는 RecursiveCharacterTextSplitter() 선언 시에 chunk_overlap을 사용하였기 때문에 텍스트를 분할할 때 겹치지 않도록 분할하는 것이 아니라 일부 겹칠 수 있도록 허용하였기 때문입니다.

이번에는 실제 챗봇과의 답변을 가정하고 사용자의 질문으로부터 챗봇의 답변이 오면 해당 챗봇의 답변으로부터 이어서 사용자가 질문하는 시나리오를 진행해보겠습니다. "전세 자금이 부족한 사람을 위한 정책 이름이 뭐야?"라는 임의의 질문을 입력합니다.

In [ ]:
input_text = "전세자금이 부족한 사람을 위한 정책 이름이 뭐야?"
llm_response = qa_chain(input_text)
get_chatbot_response(llm_response)

정책 이름은 "희망두배 청년통장"입니다.

문서 출처:
40.txt
40.txt


'희망두배 청년통장'이라는 정책이 있음을 알았습니다. 이어서 '희망두배 청년통장'의 구체적인 지원 내용을 문의합니다.

In [ ]:
query = "희망두배 청년통장은 어떤걸 지원하니?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

희망두배 청년통장은 저소득 근로 청년들의 자립을 지원하기 위해 서울시에서 운영하는 정책입니다. 이 정책은 교육비, 주거비, 결혼자금, 창업운영자금 등을 위한 저축을 지원하는 내용을 포함하고 있습니다. 신청자격은 18세에서 34세까지이며, 서울시 거주자이고 공고일 현재 근로 중인 자격 요건을 모두 충족하는 경우 신청할 수 있습니다. 자세한 내용과 신청 방법은 서울시복지재단의 홈페이지 및 서울시 안심돌봄복지과의 홈페이지에서 확인하실 수 있습니다.

문서 출처:
40.txt
36.txt


이번에는 지원 조건을 문의해봅시다.

In [ ]:
query = "희망두배 청년통장의 지원 조건은?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

희망두배 청년통장의 지원 조건은 다음과 같습니다:

1. 신청자격: 18세에서 34세까지의 청년이며, 서울시 거주자이고 공고일 현재 근로 중인 자격 요건을 모두 충족하는 경우 신청할 수 있습니다.

2. 신청 방법: 방문접수, 우편 신청(도착분 기준 18:00까지 동주민센터 도착), 이메일 신청(접수 마감일 18:00까지 도착) 방법으로 신청할 수 있습니다. 이메일로 제출할 경우 원본 스캔 또는 이미지 촬영하여 PDF, JPG, PNG 파일 형식으로 제출해야 합니다.

3. 신청 기한: 신청 기한 내에 제출서류가 누락되거나 식별이 불가능한 경우 대상자 선정에서 제외될 수 있습니다.

4. 선정 결과: 선정 결과는 추후에 공지될 예정입니다.

5. 신청 방법과 필요한 서류: 자세한 신청 방법과 필요한 서류는 서울시복지재단, 서울시 및 25개 자치구 홈페이지에서 확인하실 수 있습니다.

6. 문의 사항: 기타 문의 사항은 1688-1453, 120다산콜, 해당 주소지의 자치구 및 동주민센터로 문의하시면 됩니다.

문서 출처:
40.txt
36.txt


희망두배 청년통장의 마감 기한을 문의해봅시다.

In [ ]:
query = "희망두배 청년통장의 마감 기한은?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

희망두배 청년통장의 마감 기한은 문맥에서 명확하게 언급되지 않았습니다. 따라서 정확한 마감 기한을 알 수 없습니다. 자세한 내용은 서울시복지재단의 홈페이지나 서울시 안심돌봄복지과의 홈페이지를 참고하시기 바랍니다.

문서 출처:
40.txt
37.txt


원본 문서에서 마감 기한이 구체적으로 나와있지 않아 마감 기한을 알 수 없다고 답변합니다. 해당 챗봇은 주어진 57개의 문서를 기반으로 답변하기 때문입니다. 이렇게 다수의 문서로부터 질의 응답을 할 수 있는 챗봇을 구현해보았습니다. 이렇게 구현한 챗봇을 앞서 실습과 같이 streamlit으로 UI를 구현하거나, 카카오톡이나 텔레그램 등을 연동하여 나만의 커스텀 챗봇을 구현할 수 있습니다.

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 52.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   

To create a public link, set `share=True` in `launch()`.  
Running on https://localhost:7860/  

라는 식의 문구가 나오면 위의 localhost로 시작하는 주소를 클릭하세요.

In [ ]:
import gradio as gr

# 인터페이스를 생성.
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="청년정책챗봇") # 청년정책챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창의 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      result = qa_chain(message)
      bot_message = result['result']
      bot_message += ' # sources :'

      # 답변의 출처를 표기
      for i, doc in enumerate(result['source_documents']):
          bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + ' '

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
      chat_history.append((message, bot_message))
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행.
demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
